In [1]:
import pandas as pd
import numpy as np
import os


In [3]:
file_path = "retail_sales_dataset (1).csv"   
df = pd.read_csv(file_path)
print("Original Data Shape:", df.shape)
print(df.head())

Original Data Shape: (1000, 9)
   Transaction ID        Date Customer ID  Gender  Age Product Category  \
0               1  2023-11-24     CUST001    Male   34           Beauty   
1               2  2023-02-27     CUST002  Female   26         Clothing   
2               3  2023-01-13     CUST003    Male   50      Electronics   
3               4  2023-05-21     CUST004    Male   37         Clothing   
4               5  2023-05-06     CUST005    Male   30           Beauty   

   Quantity  Price per Unit  Total Amount  
0         3              50           150  
1         2             500          1000  
2         1              30            30  
3         1             500           500  
4         2              50           100  


In [5]:
df.describe()

,Transaction ID,Age,Quantity,Price per Unit,Total Amount
count,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000
mean,500.500000,41.39200,2.514000,179.890000,456.000000
std,288.819436,13.68143,1.132734,189.681356,559.997632
min,1.000000,18.00000,1.000000,25.000000,25.000000
25%,250.750000,29.00000,1.000000,30.000000,60.000000
50%,500.500000,42.00000,3.000000,50.000000,135.000000
75%,750.250000,53.00000,4.000000,300.000000,900.000000
max,1000.000000,64.00000,4.000000,500.000000,2000.000000


In [7]:
df.isnull().sum()

Transaction ID      0
Date                0
Customer ID         0
Gender              0
Age                 0
Product Category    0
Quantity            0
Price per Unit      0
Total Amount        0
dtype: int64

In [11]:
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

In [15]:
print(df["Date"])

0     2023-11-24
1     2023-02-27
2     2023-01-13
3     2023-05-21
4     2023-05-06
         ...    
995   2023-05-16
996   2023-11-17
997   2023-10-29
998   2023-12-05
999   2023-04-12
Name: Date, Length: 1000, dtype: datetime64[ns]


In [27]:
df = df[df["Quantity"] > 0]
df = df[df["Total Amount"] > 0]

print("\nData Shape After Cleaning:", df.shape)


Data Shape After Cleaning: (1000, 9)


In [31]:
df["year"] = df["Date"].dt.year
df["month"] = df["Date"].dt.month
df["year_month"] = df["Date"].dt.to_period("M").dt.to_timestamp()

In [33]:
print(df["year"])

0      2023
1      2023
2      2023
3      2023
4      2023
       ... 
995    2023
996    2023
997    2023
998    2023
999    2023
Name: year, Length: 1000, dtype: int32


In [35]:
bins = [0, 18, 25, 35, 45, 60, 120]
labels = ["<18", "18-24", "25-34", "35-44", "45-59", "60+"]
df["age_group"] = pd.cut(df["Age"], bins=bins, labels=labels, right=False)

In [37]:
print(df["age_group"])

0      25-34
1      25-34
2      45-59
3      35-44
4      25-34
       ...  
995      60+
996    45-59
997    18-24
998    35-44
999    45-59
Name: age_group, Length: 1000, dtype: category
Categories (6, object): ['<18' < '18-24' < '25-34' < '35-44' < '45-59' < '60+']


In [39]:
output_dir = "retail_project_output"
os.makedirs(output_dir, exist_ok=True)

In [41]:
df.to_csv(f"{output_dir}/sales_cleaned.csv", index=False)

In [43]:
monthly_sales = (
    df.groupby("year_month")
    .agg(
        total_revenue=("Total Amount", "sum"),
        total_quantity=("Quantity", "sum"),
        num_transactions=("Transaction ID", "nunique"),
        unique_customers=("Customer ID", "nunique"),
    )
    .reset_index()
    .sort_values("year_month")
)

In [45]:
monthly_sales.to_csv(f"{output_dir}/monthly_sales.csv", index=False)

In [49]:
monthly_sales.tail()

,year_month,total_revenue,total_quantity,num_transactions,unique_customers
8,2023-09-01,23620,170,65,65
9,2023-10-01,46580,252,96,96
10,2023-11-01,34920,205,78,78
11,2023-12-01,44690,207,91,91
12,2024-01-01,1530,4,2,2


In [51]:
category_sales = (
    df.groupby("Product Category")
    .agg(
        total_revenue=("Total Amount", "sum"),
        total_quantity=("Quantity", "sum"),
        num_transactions=("Transaction ID", "nunique"),
        unique_customers=("Customer ID", "nunique"),
    )
    .reset_index()
    .sort_values("total_revenue", ascending=False)
)

category_sales.to_csv(f"{output_dir}/category_sales.csv", index=False)

In [55]:
gender_sales = (
    df.groupby("Gender")
    .agg(
        total_revenue=("Total Amount", "sum"),
        total_quantity=("Quantity", "sum"),
        num_customers=("Customer ID", "nunique"),
    )
    .reset_index()
)

gender_sales.to_csv(f"{output_dir}/gender_sales.csv", index=False)

In [57]:
gender_sales

,Gender,total_revenue,total_quantity,num_customers
0,Female,232840,1298,510
1,Male,223160,1216,490


In [68]:
age_sales = (
    df.groupby("age_group",observed=False)
    .agg(
        total_revenue=("Total Amount", "sum"),
        total_quantity=("Quantity", "sum"),
        num_customers=("Customer ID", "nunique"),
    )
    .reset_index()
)

age_sales.to_csv(f"{output_dir}/age_group_sales.csv", index=False)

In [70]:
age_sales

,age_group,total_revenue,total_quantity,num_customers
0,<18,0,0,0
1,18-24,74650,366,149
2,25-34,97090,522,203
3,35-44,96835,533,207
4,45-59,142610,812,326
5,60+,44815,281,115


In [74]:
snapshot_date = df["Date"].max() + pd.Timedelta(days=1)

rfm = (
    df.groupby("Customer ID")
    .agg(
        recency=("Date", lambda x: (snapshot_date - x.max()).days),
        frequency=("Transaction ID", "nunique"),
        monetary=("Total Amount", "sum"),
    )
    .reset_index()
)

rfm["monetary"] = rfm["monetary"].round(2)

rfm.to_csv(f"{output_dir}/customer_rfm.csv", index=False)

print("\nETL Completed Successfully!")
print("All output files saved in:", output_dir)


ETL Completed Successfully!
All output files saved in: retail_project_output


In [76]:
rfm

,Customer ID,recency,frequency,monetary
0,CUST001,39,1,150
1,CUST002,309,1,1000
2,CUST003,354,1,30
3,CUST004,226,1,500
4,CUST005,241,1,100
...,...,...,...,...
995,CUST995,247,1,30
996,CUST996,231,1,50
997,CUST997,46,1,90
998,CUST998,65,1,100
